Importo las librerias que estare utilizando

In [1]:
import pandas as pd
import numpy as np
import ast

Comienzo las cargas de archivos a un dataframe

In [2]:
reviews = []
with open('data/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_reviews = pd.DataFrame(reviews)

In [3]:
items = []
with open('data/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_items = pd.DataFrame(items)

In [4]:
import json

# Lista para almacenar los objetos JSON 
user_games = []

# Abre el archivo JSON y carga su contenido en un diccionario
with open('data/output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        try:
            json_data = json.loads(line)
            user_games.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en línea: {e}")

# Crear el DataFrame anidado
df_games = pd.DataFrame(user_games)

Aqui Comenzare con el EDA

Comenzare con **df_reviews**

Lo primero que hare sera desanidar reviews para poder hacer un mejor analisis de los datos.

In [5]:
df_reviews_new = df_reviews.explode('reviews')

In [6]:
df_reviews_new = pd.concat([df_reviews_new.drop(['reviews'], axis=1), df_reviews_new['reviews'].apply(pd.Series)], axis=1)

Luego eliminare unas columnas las cuales no usaré.

In [7]:
df_reviews_new.drop(columns=['user_url','posted','funny','last_edited','helpful',0],axis=1,inplace=True)

Luego de esto, haré un analisis de datos para verificar si hay datos faltantes, para esto utilizare la extencion de vscode que se llama 'Data Wrangler' y anotare aqui los resultados:


Al realizar el analisis, note que hay 28 valores nulos en 4 columnas (posted,item_id,recomendend,review) y ahora realizare un dropna para eliminar esos valores faltantes.

In [8]:
df_reviews_new.dropna(inplace=True)

Por ultimo cambiare el nombre de la columna item_id por id_game.

In [9]:
df_reviews_new.rename({'item_id':'id'},axis=1,inplace=True)

Ahora seguire con **df_items**

Lo primero que hare sera desanidar la columna items para poder hacer un mejor analisis de datos.

In [10]:
df_items_explode = df_items.explode('items')

In [11]:
df_items_explode.reset_index(inplace=True)

In [12]:
df_items_desanidados = pd.json_normalize(df_items_explode['items'])

In [13]:
df_items_defined = pd.concat([df_items_explode,df_items_desanidados],axis=1)

In [14]:
df_items_defined.drop(columns=['index','items','items_count','steam_id','user_url','playtime_2weeks'],axis=1,inplace=True)

ya teniendo nuestro dataframe completamente desanidado, procedere a eliminar datos nulos basandome en las columnas item_id y item_name.

In [15]:
df_items_defined.dropna(subset=['item_id','item_name'],inplace=True)

Por ultimo cambiare el nombre de algunas columnas

In [16]:
df_items_defined.rename({'item_id':'id','item_name':'game'},inplace=True,axis=1)

Terminare con **df_games**

Lo primero que hice fue eliminar las primeras 88310 filas porque eran valores nulos y luego hice un reset_index.

In [18]:
df_games.drop(df_games.index[0:88310],inplace=True)

In [19]:
df_games.reset_index(inplace=True,drop=True)

Aqui hare una copia antes de eliminar **Specs y tags** para luego trabajar con mis datos que usare para el sistema de recomendación.

In [20]:
games = df_games[['genres','app_name','release_date','tags','specs','price','id','developer']]

Elimine las columnas que no estaria usando.

In [21]:
df_games.drop(columns=['url','specs','tags','reviews_url','early_access'],axis=1,inplace=True)

Eliminare las filas vacias y dejare solo el año en la columna **release_date**.

In [22]:
df_games['release_date']=pd.to_datetime(df_games['release_date'],errors="coerce").dt.year

In [23]:
df_games.dropna(subset=['release_date'], inplace=True)

In [24]:
df_games['release_date'] = df_games['release_date'].astype(int).astype(str)

cambiare el nombre de dos columnas 

In [25]:
df_games.rename({'app_name':'game'},axis=1,inplace=True)

reemplazare los valores string en la columna de price que representan que son gratis por el 0.

In [26]:
replace_dict = {
    'Free To Play': 0,
    'Free to Play': 0,
    'Free': 0,
    'Free Demo': 0,
    'Play for Free!': 0,
    'Install Now': 0,
    'Play WARMACHINE: Tactics Demo': 0,
    'Free Mod': 0,
    'Third-party': 0,
    'Play Now': 0,
    'Free HITMAN™ Holiday Pack': 0,
    'Play the Demo': 0,
    'Install Theme':0
}
df_games['price'].replace(replace_dict, inplace=True)

**Aqui comenzare con mi dataframe games para el modelo de recomendación.**

hare un dropna para eliminar filas vacias en mi dataframe.

In [ ]:
games.dropna(inplace=True)

C:\Users\Win10\AppData\Local\Temp\ipykernel_15952\1030695128.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games.dropna(inplace=True)


Comenzare desanidando **genres**.

In [ ]:
games = games.explode('genres')

In [ ]:
games.dropna(subset=['genres'],inplace=True)

In [ ]:
games.drop_duplicates(subset='id',inplace=True)

Ahora hare lo mismo con **specs**.

In [ ]:
games = games.explode('specs')

In [ ]:
games.drop_duplicates(subset='id',inplace=True)

Ahora hare lo mismo con **tags**.

In [ ]:
games = games.explode('tags')

In [ ]:
games.drop_duplicates(subset='id',inplace=True)

Reseteo el indice.

In [ ]:
games.reset_index(inplace=True,drop=True)

Hare una estandarizacion y una clasificacion de la columna price.

In [ ]:
replace_dict = {
    'Free To Play': 0,
    'Free to Play': 0,
    'Free': 0,
    'Free Demo': 0,
    'Play for Free!': 0,
    'Install Now': 0,
    'Play WARMACHINE: Tactics Demo': 0,
    'Free Mod': 0,
    'Third-party': 0,
    'Play Now': 0,
    'Free HITMAN™ Holiday Pack': 0,
    'Play the Demo': 0,
    'Install Theme':0
}
games['price'].replace(replace_dict, inplace=True)

Luego de estandarizar la columna price, comenzaré con la clasificacion.

Obtengo la mediana de los precios y etiqueto en base a la mediana.

In [ ]:
median_price = games['price'].median()
median_price

4.99

In [ ]:
def etiquetar_precio(valor):
    if valor == 0:
        return 'gratis'
    elif valor <= 4.99:
        return "barato"
    else:
        return "caro"

In [ ]:
games['price'] = games['price'].apply(etiquetar_precio)

Paso la estandarizacion a numeros.

In [ ]:
price = {'gratis':0,'barato':1,'caro':2}

In [ ]:
games['price'].replace(price,inplace=True)

Luego cambio los nombres de las columnas app_name y id.

In [ ]:
games.rename({'app_name':'game'},axis=1,inplace=True)

Haré una clasificacion numerica a las columnas que estare mostrando para tener un mejor resultado en el modelo de recomendacion.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_genres = LabelEncoder()
le_date = LabelEncoder()
le_specs = LabelEncoder()
le_tags = LabelEncoder()
le_developer = LabelEncoder()

In [ ]:
games['genres'] = le_genres.fit_transform(games['genres'])
games['release_date'] = le_date.fit_transform(games['release_date'])
games['specs'] = le_specs.fit_transform(games['specs'])
games['tags'] = le_tags.fit_transform(games['tags'])
games['developer'] = le_developer.fit_transform(games['developer'])                     

Por ultimo creare un csv para cada dataframe con el cual estuve trabajando.

In [ ]:
df_reviews_new.to_csv('nueva data/reviews.csv',index=False)

In [ ]:
df_games.to_csv('nueva data/games.csv',index=False)

In [ ]:
games.to_csv('nueva data/games_model.csv',index=False)

Para el archivo items lo que hara ser comprimirlo a en un archivo parquet.

Importamos las librerias que necesitaremos.

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:
# Convierte el DataFrame de pandas a una tabla de pyarrow
table = pa.Table.from_pandas(df_items_defined)

# Define el nombre de tu archivo Parquet de destino
archivo_parquet = 'nueva data/items_parquet.parquet'

# Escribe la tabla en formato Parquet
pq.write_table(table, archivo_parquet)

Aqui terminaria mi EDA, las tranformaciones de los dataframes y carga de archivos.